In [1]:
import numpy as np
import random
import pandas as pd

import sys
sys.path.append('C:\\Users\\PC\\Documents\\GitHub\\Ugur-dynamic-scheduling\\functions')
import vizings_schedule
from yeni_model import yeni_model
from weighted_days_model import weighted_days_model
from generate_results import generate_results


In [2]:
big_list_v_min = []
big_list_v_max = []
big_list_c_min = []
big_list_c_max = []
big_list_m = []
n=6
teams = list(range(1,n+1))
days = list(range(1,n))
#results_df = pd.read_excel('C:\\Users\\PC\\Desktop\\match_resultlar.xlsx', sheet_name='Germany second half')
results_df = generate_results(teams)
result_array = np.zeros((n, n), dtype=int)
for _, row in results_df.iterrows():
    result = row['Result']
    team1 = row['Team 1']
    team2 = row['Team 2']

    index_team1 = teams.index(team1)
    index_team2 = teams.index(team2)

    if team1 == result:
        result_array[index_team1, index_team2] = 3
    elif team2 == result:
        result_array[index_team2, index_team1] = 3
    else:
        result_array[index_team1, index_team2] = 1
        result_array[index_team2, index_team1] = 1
result_matrix = result_array

number_of_matches = int((n*(n-1))/2)
print(result_matrix)
simulation = []
canon_simulation = []

data = pd.DataFrame(result_array, columns=teams, index=teams)

output, d1_df, d2_df, y_df, z_df, obj_value, l, l2= weighted_days_model(teams, days, data)
sonuc = np.array(output)
ratings = {i: 0 for i in teams}

objective_values = []


for d in days:
    obj_count =0
    for row in sonuc:
        day,team1,team2 = row
        if d==day:
            obj_count +=(abs(ratings[team1] - ratings[team2]))
            ratings[team1] += result_matrix[team1-1,team2-1]
            ratings[team2] += result_matrix[team2-1,team1-1]
    print(ratings)
    objective_values.append(obj_count)
a = sum(objective_values)
big_list_m.append(a)


output2, d1_df2, d2_df2, y_df2, z_df2, obj_value2, li, li2= yeni_model(teams, days, data)
sonuc2 = np.array(output2)
ratings = {i: 0 for i in teams}

objective_values = []


for d in days:
    obj_count =0
    for row in sonuc2:
        day,team1,team2 = row
        if d==day:
            obj_count +=(abs(ratings[team1] - ratings[team2]))
            ratings[team1] += result_matrix[team1-1,team2-1]
            ratings[team2] += result_matrix[team2-1,team1-1]
    print(ratings)
    objective_values.append(obj_count)
b = sum(objective_values)
big_list_m.append(b)


[[0 3 3 3 3 3]
 [0 0 3 3 3 3]
 [0 0 0 3 3 3]
 [0 0 0 0 3 3]
 [0 0 0 0 0 3]
 [0 0 0 0 0 0]]
0.2
0.25
0.3333333333333333
0.5
1.0
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-08
Read LP format model from file C:\Users\PC\AppData\Local\Temp\tmprkepghx5.pyomo.lp
Reading time = 0.01 seconds
x1: 570 rows, 375 columns, 2025 nonzeros
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 570 rows, 375 columns and 2025 nonzeros
Model fingerprint: 0x6fe1b9b7
Variable types: 150 continuous, 225 integer (75 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 388 rows 

C:\Users\PC\Documents\GitHub\Ugur-dynamic-scheduling\functions\weighted_days_model.py:172: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  l2 = pd.concat([l2, new_row1]).reset_index(drop=True)
C:\Users\PC\Documents\GitHub\Ugur-dynamic-scheduling\functions\weighted_days_model.py:187: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  d1_df = pd.concat([d1_df, new_row1]).reset_index(drop=True)
C:\Users\PC\Documents\GitHub\Ugur-dynamic-scheduling\functions\weighted_days_model.py:199: FutureWarning: The be

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-08
Read LP format model from file C:\Users\PC\AppData\Local\Temp\tmpr8ax9vvq.pyomo.lp
Reading time = 0.01 seconds
x1: 570 rows, 375 columns, 2025 nonzeros
Set parameter TimeLimit to value 2400
Set parameter PoolSearchMode to value 2
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 570 rows, 375 columns and 2025 nonzeros
Model fingerprint: 0x0bc953ab
Variable types: 150 continuous, 225 integer (75 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 23 rows and 23 columns
Presolve time: 0.00s
Presolved: 547 rows, 352 columns, 1986 nonzeros
Vari

C:\Users\PC\Documents\GitHub\Ugur-dynamic-scheduling\functions\yeni_model.py:164: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  l2 = pd.concat([l2, new_row1.dropna()]).reset_index(drop=True)
C:\Users\PC\Documents\GitHub\Ugur-dynamic-scheduling\functions\yeni_model.py:180: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  d1_df = pd.concat([d1_df, new_row1]).reset_index(drop=True)
C:\Users\PC\Documents\GitHub\Ugur-dynamic-scheduling\functions\yeni_model.py:192: FutureWarning: The behavior of DataFram

In [3]:
print(big_list_m)
print(obj_value)

[30, 30]
11.75


path = ('C:\\Users\\PC\\Documents\\GitHub\\Ugur-dynamic-scheduling\\weighted_6takim.xlsx')
with pd.ExcelWriter(path, engine='xlsxwriter') as excel_writer:


# Write each DataFrame to a different sheet
    output.to_excel(excel_writer, sheet_name='schedule', index=False)
    d1_df.to_excel(excel_writer, sheet_name='d1', index=False)
    d2_df.to_excel(excel_writer, sheet_name='d2', index=False)
    y_df.to_excel(excel_writer, sheet_name='y', index=False)
    z_df.to_excel(excel_writer, sheet_name='z', index=False)


path2 = ('C:\\Users\\PC\\Documents\\GitHub\\Ugur-dynamic-scheduling\\unweighted_6takim.xlsx')
with pd.ExcelWriter(path2, engine='xlsxwriter') as excel_writer:


# Write each DataFrame to a different sheet
    output2.to_excel(excel_writer, sheet_name='schedule', index=False)
    d1_df2.to_excel(excel_writer, sheet_name='d1', index=False)
    d2_df2.to_excel(excel_writer, sheet_name='d2', index=False)
    y_df2.to_excel(excel_writer, sheet_name='y', index=False)
    z_df2.to_excel(excel_writer, sheet_name='z', index=False)

